In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_ds = datasets.FashionMNIST(
    './data', 
    train=True, 
    download=True, 
    transform=transforms.ToTensor()
    )
    
test_ds = datasets.FashionMNIST(
    './data', 
    train=False, 
    download=True, 
    transform=transforms.ToTensor())

In [3]:
batch_size = 64

train_dl = DataLoader(
    train_ds,
    batch_size = batch_size,
    shuffle=True
)

test_dl = DataLoader(
    test_ds,
    batch_size = batch_size,
)

In [4]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        # self.flatten = nn.Flatten(),
        # self.l1 = nn.Linear(28 * 28, 512),
        # self.relu = nn.ReLU(),
        # self.l2 = nn.Linear(512,512),
        # self.l3= nn.Linear(512,10)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        # x = self.flatten(x)
        # x1 = x.view(-1, 28*28)
        # out1 = self.relu(self.l1(x1))
        # out2 = self.relu(self.l2(out1))
        # out3 = self.l3(out2)
        # return out3
        x = x.view(-1, 28*28)
        x = self.linear_relu_stack(x)
        return x

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model = NN().to(device)
model

NN(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [8]:
epochs = 10

model.train()
for epoch in range(epochs):
    
    for batch, (images, labels) in enumerate(train_dl):
        images, labels = images.to(device), labels.to(device)

        # predict the batch of images and calculate the loss
        pred = model(images)
        loss = loss_fn(pred, labels)

        # calculate the gradients and update the parameters (backpropogation)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch+1) % 200 == 0:
            print(f"Epoch no :{epoch+1}, step no {batch+1}, loss is: {loss.item():.4f}")

Epoch no :1, step no 200, loss is: 2.2952
Epoch no :1, step no 400, loss is: 2.2641
Epoch no :1, step no 600, loss is: 2.2388
Epoch no :1, step no 800, loss is: 2.2030
Epoch no :2, step no 200, loss is: 2.1591
Epoch no :2, step no 400, loss is: 2.1009
Epoch no :2, step no 600, loss is: 2.0119
Epoch no :2, step no 800, loss is: 1.9982
Epoch no :3, step no 200, loss is: 1.8866
Epoch no :3, step no 400, loss is: 1.7847
Epoch no :3, step no 600, loss is: 1.5818
Epoch no :3, step no 800, loss is: 1.5210
Epoch no :4, step no 200, loss is: 1.4872
Epoch no :4, step no 400, loss is: 1.3255
Epoch no :4, step no 600, loss is: 1.3065
Epoch no :4, step no 800, loss is: 1.3051
Epoch no :5, step no 200, loss is: 1.2359
Epoch no :5, step no 400, loss is: 1.1630
Epoch no :5, step no 600, loss is: 1.1217
Epoch no :5, step no 800, loss is: 1.0989
Epoch no :6, step no 200, loss is: 1.1131
Epoch no :6, step no 400, loss is: 1.0207
Epoch no :6, step no 600, loss is: 0.9519
Epoch no :6, step no 800, loss is:

In [9]:
num_correct = 0
no_samples = 0
model.eval()

with torch.no_grad():
    for images, labels in test_dl:
        images, labels =images.to(device), labels.to(device)

        scores = model(images)
        _, predictions = torch.max(scores, -1)
        num_correct += (predictions == labels).sum().item()
        no_samples += predictions.size(-1)

    accuracy = 100 * num_correct / no_samples
    print(f"The accuracy of the model is {accuracy:.2f}%")

The accuracy of the model is 71.02%
